In [41]:
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import cv2
# Load a pre-trained YOLOv8 model
model = YOLO('./models/best.pt')
tracker = DeepSort(max_age=30, n_init=15, max_cosine_distance=0.15, bgr=True)
video_path = './Assignment Materials/Assignment Materials/15sec_input_720p.mp4'

In [1]:
!nvidia-smi

Tue Jun 24 01:03:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.14                 Driver Version: 566.14         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti   WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   68C    P8              5W /   50W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [42]:
import sys
# Check if the model is loaded correctly
if model is None:
    print("Error: Model not loaded.")
    sys.exit(1)

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("End of video or error reading frame.")
        break

    results = model(frame)
    boxes = results[0].boxes if results else None

    detections = []

    print("Gathering detections...")
    if boxes is None  or len(boxes) == 0:
        print("No detections found.")
        continue
    for box in boxes:
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        w, h = x2 - x1, y2 - y1
        cls = box.cls[0]
        if cls != 2:
            continue
        conf = box.conf[0]
        detections.append([(x1, y1, w, h), cls, conf])
        
        # Update Deep SORT tracker
    tracks = tracker.update_tracks(detections, frame=frame)

    # Draw tracks
    for track in tracks:
        if not track.is_confirmed():
            continue
        ltrb = track.to_ltrb()
        track_id = track.track_id
        cv2.rectangle(frame, (int(ltrb[0]), int(ltrb[1])), (int(
            ltrb[2]), int(ltrb[3])), (0, 0, 255), 2)
        cv2.putText(frame, f"{str(track_id)}", (int(ltrb[0]), int(
            ltrb[1] - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow('YOLOv8 Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Exiting video stream.")
        break

cv2.destroyAllWindows()
cap.release()

print("Detections gathered, tracking...")


0: 384x640 1 ball, 16 players, 2 referees, 76.5ms
Speed: 3.1ms preprocess, 76.5ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)
Gathering detections...

0: 384x640 18 players, 2 referees, 64.6ms
Speed: 2.8ms preprocess, 64.6ms inference, 5.9ms postprocess per image at shape (1, 3, 384, 640)
Gathering detections...

0: 384x640 1 ball, 16 players, 2 referees, 63.1ms
Speed: 3.3ms preprocess, 63.1ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)
Gathering detections...

0: 384x640 1 ball, 14 players, 2 referees, 62.9ms
Speed: 2.0ms preprocess, 62.9ms inference, 5.6ms postprocess per image at shape (1, 3, 384, 640)
Gathering detections...

0: 384x640 1 ball, 14 players, 2 referees, 64.1ms
Speed: 2.6ms preprocess, 64.1ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)
Gathering detections...

0: 384x640 1 ball, 16 players, 2 referees, 63.4ms
Speed: 2.7ms preprocess, 63.4ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)
